In [ ]:
#Tensorflow
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, activations
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from sklearn.ensemble import RandomForestRegressor

#Pandas and numpy for data formats
import pandas as pd
import numpy as np


#glob for data import
import glob
import random
#PyTorch
import torch
import torch.nn as nn
#SK Learn
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
#from sklearn.utils import shuffle
import math
import sys
#MatPlotLib
import matplotlib.pyplot as plt

from models import simpleRegression
from models import feedForward, sequentialModel, sequentialDropout, lstmModel, cnnModel, cnnModelFilters, scoringSystem

#import functions
from dataImport import importTrainingData, importParamsData, importTestData, importTrainParameter, importTargetParameter, importEvalParameter
from preprocessing import preprocessData, createTrainTensors, createParamsTensors, preprocessParams, rescaleData, preprocessParameters


#importTrainData()
train_data = np.load("../train_40000.npy")
params_data = np.load("../train_target_40000.npy")
train_parameter = np.load("../train_parameter_40000.npy")
#
validation_data = np.load("../val_train_40000.npy")
validation_params = np.load("../val_target_40000.npy")
validation_parameter = np.load("../val_parameter_40000.npy")

#Create stats for zScore
mean_value_target = np.mean(params_data)
standard_deviation_target = np.std(params_data)

mean_value = np.mean(train_data)
standard_deviation = np.std(train_data)
measurements_to_remove = 30


median_train = False
replaceValuesBiggerOne_train=False
replaceFirstXValues_train=False
removeFirstXValues_train=True
normalizeData_train=False
zScore_train=True

train_data = preprocessData(train_data,
                          median=median_train,
                          replaceValuesBiggerOne = replaceValuesBiggerOne_train,
                          replaceFirstXValues = replaceFirstXValues_train,
                          removeFirstXValues = removeFirstXValues_train,
                          normalizeData = normalizeData_train,
                          zScore = zScore_train,
                          zScore_mean = mean_value,
                          zScore_sd = standard_deviation,
                          number_to_remove = measurements_to_remove)

train_target = preprocessData(params_data,
                          median=False,
                          replaceValuesBiggerOne=False,
                          replaceFirstXValues=False,
                          removeFirstXValues=False,
                          normalizeData=False,
                          zScore=False,
                          zScore_mean=mean_value_target,
                          zScore_sd=standard_deviation_target,
                          number_to_remove=measurements_to_remove)

validation_data = preprocessData(validation_data,
                          median = median_train,
                          replaceValuesBiggerOne = replaceValuesBiggerOne_train,
                          replaceFirstXValues = replaceFirstXValues_train,
                          removeFirstXValues = removeFirstXValues_train,
                          normalizeData = normalizeData_train,
                          zScore = zScore_train,
                          zScore_mean = mean_value,
                          zScore_sd = standard_deviation,
                          number_to_remove = measurements_to_remove)

validation_target = preprocessData(validation_params,
                          median=False,
                          replaceValuesBiggerOne=False,
                          replaceFirstXValues=False,
                          removeFirstXValues=False,
                          normalizeData=False,
                          zScore=False,
                          zScore_mean=mean_value_target,
                          zScore_sd=standard_deviation_target,
                          number_to_remove=measurements_to_remove)


number_of_measurements = 300
if removeFirstXValues_train == True:
    number_of_measurements = 300 - measurements_to_remove    
    
#reshaping train data
if type(train_data) != np.ndarray:
    train_data = train_data.to_numpy().reshape(-1, 55, number_of_measurements)
else:
    train_data = train_data.reshape(-1, 55, number_of_measurements)

#reshaping target data
if type(params_data) != np.ndarray:
    params_data = params_data.to_numpy()

#reshaping val data
if type(validation_data) != np.ndarray:
    validation_data = validation_data.to_numpy().reshape(-1, 55, number_of_measurements)
else:    
    validation_data = validation_data.reshape(-1, 55, number_of_measurements)

#reshaping val target
if type(validation_params) != np.ndarray:
    validation_params = validation_params.to_numpy()

    
def scaleToMaxToOne(dat, min, max):
    scaled_dat = dat / max
    return scaled_dat

def unscaleMaxToOne(scaled_dat, min, max):
    unscaled_dat = scaled_dat * max
    return unscaled_dat

min_target = np.min(train_target) 
max_target = np.max(train_target)
print(f"Rescaling targets with max = {max_target}")
train_target = scaleToMaxToOne(train_target, min_target, max_target)
val_target = scaleToMaxToOne(validation_target, min_target, max_target)    

prevent_overfitting = keras.callbacks.EarlyStopping(monitor="val_loss", patience = 20, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, min_delta=0.0001)

In [ ]:
train_data_reshaped = train_data.reshape(2200000, 270)

In [ ]:
train_target_reshaped = train_target.reshape(2200000)

In [ ]:
validation_data_reshaped = validation_data.reshape(550000, 270)

In [ ]:
rfr = RandomForestRegressor(n_estimators = 100, verbose=1, n_jobs = -1)

rfrModel = rfr.fit(train_data_reshaped, train_target_reshaped)
prediction = rfrModel.predict(validation_data)